# Setup and test algebraic eqn solver

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import os
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
os.chdir(sys.path[0] + '\..') # change our working directory to the root of the pybamm folder
import pybamm
print(pybamm.__path__[0])
%matplotlib widget

c:\Users\Vivian\Dropbox (University of Michigan)\from_box\Research\PyBaMM\PyBaMM\pybamm


In [14]:
class PouchVenting(pybamm.BaseSubModel):
    def __init__(self, param, options = None):
        super().__init__(param)

    def get_fundamental_variables(self):
        delta_sigma = pybamm.Variable("Cell expansion stress [kPa]", domain="current collector")

        variables = {
            "Cell expansion stress [kPa]": delta_sigma,
        }
        return variables

    def get_coupled_variables(self, variables):
        param = self.param
        m_an = param.therm.m_an
        x_sei_0 = param.therm.x_sei_0
        M_C6 = param.therm.M_an
        V_head_0 = param.therm.V_head_0
        A_surf = param.therm.A_surf
        L = param.therm.L_poron
        E = param.therm.E_poron
        alpha_cell = param.therm.alpha_cell
        P_crit = param.therm.P_crit
        sigma_0 = param.therm.sigma_0
        P_atm = param.therm.P_atm

        x_sei = variables["Fraction of Li in SEI"]
        delta_sigma = variables["Cell expansion stress [kPa]"]
        T = variables["X-averaged negative electrode temperature"]
        T_dimensional = param.Delta_T * T + param.T_ref
        delta_T = param.Delta_T * T

        delta_d = L/E*delta_sigma
        V_head = V_head_0 + A_surf*(delta_d-alpha_cell*delta_T)   
        P_sat = param.therm.P_sat(T) #dimensionless
        n_CO2 = m_an*(x_sei_0 - x_sei)/(2*M_C6)
        P_CO2 = (n_CO2*constants.R*T/V_head/1000)/P_crit #kPa
        P_total = P_CO2 + P_sat 

        delta_sigma_gas = P_total-sigma_0-P_atm
        dleta_sigma_themal_expansion = E*alpha_cell*delta_T/L

        
        variables = {
            "Electrolyte gas saturation pressure": P_sat/P_crit,
            "CO2 gas pressure": P_CO2/P_crit,
            "Total gas pressure [kPa]": P_total/P_crit,
            "Electrolyte gas saturation pressure [kPa]": P_sat,
            "CO2 gas pressure [kPa]": P_CO2,
            "Total gas pressure [kPa]": P_total,
            "Headspace volume [m3]": V_head,

            
        }

        return variables

    def set_algebraic(self, variables):
        param = self.param
        P_crit = param.therm.P_crit
        L = param.therm.L_poron
        E = param.therm.E_poron
        alpha_cell = param.therm.alpha_cell
        P_atm = param.therm.P_atm
        sigma_0 = param.therm.sigma_0
    
        T = variables["X-averaged negative electrode temperature"]
        delta_T = param.Delta_T * T
        delta_sigma = variables["Cell expansion stress [kPa]"]
        P_total = variables["Total gas pressure [kPa]"]        
        self.algebraic = {delta_sigma: pybamm.maximum(P_total-sigma_0-P_atm, E*alpha_cell*delta_T/L)} #pybamm.maximum(P_total-sigma_0-P_atm, E*alpha_cell*delta_T/L)

    def set_initial_conditions(self, variables):
        # param = self.param
        # sigma_0 = param.therm.sigma_0
        delta_sigma = variables["Cell expansion stress [kPa]"]
        self.initial_conditions = {
            delta_sigma: pybamm.Scalar(0), 
            }

In [9]:
# load data import ESC data from file
raw_data = pd.read_csv("./fast_discharge/ESC_100SOC_full.csv")
esc_start = raw_data[-raw_data['Current Shunt']>1].index[0]
# esc_end = raw_data[raw_data['Time (s)']>15*60].index[0]
esc_end =len(raw_data)
# T_amb = np.mean(raw_data['Cell Temperature'][(raw_data.index < esc_start) & (raw_data['Cell Temperature'] >0)])
data = raw_data[['Time (s)', 'Voltage (V)', 'Cell Temperature', 'Current Shunt', 'Force']].loc[esc_start:esc_end].copy()
data['Current Shunt'] = -data['Current Shunt']
data['Time (s)'] = data['Time (s)'] - data['Time (s)'].loc[esc_start]
df_labels = ['t', 'V', 'Temp','I', 'F']
data.set_axis(df_labels, axis=1, inplace=True)

data.Temp = data.Temp + 273.15

temperature_profile = data[['t','Temp']]

In [36]:
# options = {
#     # 'thermal': pybamm.thermal.Lumped(param),
#     'venting': pybamm.venting.PouchVenting(param), 
#     # 'decomposition': pybamm.decomposition.SeiDecomposition(param),
#     # "external submodels": ["thermal"]
# }
options = {
    # 'thermal': 'x-lumped',
    # 'venting': 'true', 
    'decomposition': 'true',
    # "external submodels": ['thermal']
}
# model = pybamm.lithium_ion.SPMe(options = options)


In [54]:
# model = pybamm.()
chemistry = pybamm.parameter_sets.Tran2023
param = pybamm.ParameterValues(chemistry = chemistry)
parameters = pybamm.LithiumIonParameters()

model = pybamm.BaseModel(options)
model.submodels = {
    # 'thermal': pybamm.thermal.Lumped(parzameters),
    'venting': pybamm.venting.PouchVenting(parameters), 
    'Decomposition': pybamm.decomposition.SeiDecomposition(parameters),
    "external submodels": [pybamm.thermal.Lumped(parameters)],
}
model.build_model()

AttributeError: 'list' object has no attribute 'get_fundamental_variables'

In [47]:
# set up sim
t_end = 300
t_eval = np.linspace(0,t_end,t_end*10)


solver = pybamm.CasadiSolver()
sim = pybamm.Simulation(model, parameter_values=param)

# define temperature change with time
T_ref = param.evaluate(parameters.T_ref)
Delta_T = param.evaluate(parameters.Delta_T)
T_av_dim = interp1d(temperature_profile.t, temperature_profile.T, kind = 'linear')
for i in np.arange(1, len(t_eval) - 1): 
    dt = t_eval[i + 1] - t_eval[i]
    T_av = (T_av_dim(t_eval[i]) - T_ref) / Delta_T
    external_variables = {
        "X-averaged negative electrode temperature": T_av, 
        }
    solution = sim.step(dt, external_variables=external_variables)

output_variables =[
    "X-averaged negative electrode temperature",
    "Fraction of Li in SEI",
    "Electrolyte gas saturation pressure [kPa]"
    "CO2 gas pressure [kPa]"
    "Cell expansion stress [kPa]"
    "Headspace volume [m3]"

]

sim.plot(output_variables)

KeyError: 'current collector'